In [ ]:
!nvidia-smi

Thu Mar 23 07:09:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/My Drive/EVA8_S11_Course_Docs/BERT" "."

In [ ]:
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re

In [ ]:
# =============================================================================
# Transformer
# =============================================================================
def attention(q, k, v, mask = None, dropout = None):
    scores = q.matmul(k.transpose(-2, -1))
    scores /= math.sqrt(q.shape[-1])

    #mask
    scores = scores if mask is None else scores.masked_fill(mask == 0, -1e3)

    scores = F.softmax(scores, dim = -1)
    scores = dropout(scores) if dropout is not None else scores
    output = scores.matmul(v)
    print(f' Attention output : {output.shape} v : {v.shape} scores : {scores.shape}')
    return output

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, out_dim, dropout=0.1):
        super().__init__()

#        self.q_linear = nn.Linear(out_dim, out_dim)
#        self.k_linear = nn.Linear(out_dim, out_dim)
#        self.v_linear = nn.Linear(out_dim, out_dim)
        self.linear = nn.Linear(out_dim, out_dim*3)

        self.n_heads = n_heads
        self.out_dim = out_dim
        self.out_dim_per_head = out_dim // n_heads
        self.out = nn.Linear(out_dim, out_dim)
        self.dropout = nn.Dropout(dropout)

    def split_heads(self, t):
        print(f'MHA : split_heads t.shape {t.shape} self.n_heads {self.n_heads} self.out_dim_per_head {self.out_dim_per_head}')
        return t.reshape(t.shape[0], -1, self.n_heads, self.out_dim_per_head)

    def forward(self, x, y=None, mask=None):
        #in decoder, y comes from encoder. In encoder, y=x
        y = x if y is None else y

        qkv = self.linear(x) # BS * SEQ_LEN * (3*EMBED_SIZE_L)
        print(f'MHA : x.shape {x.shape} , qkv.shape {qkv.shape}')
        q = qkv[:, :, :self.out_dim] # BS * SEQ_LEN * EMBED_SIZE_L
        print(f'MHA : q.shape {q.shape} , self.out_dim {self.out_dim}')
        k = qkv[:, :, self.out_dim:self.out_dim*2] # BS * SEQ_LEN * EMBED_SIZE_L
        print(f'MHA : k.shape {k.shape} , self.out_dim*2 {self.out_dim*2}')
        v = qkv[:, :, self.out_dim*2:] # BS * SEQ_LEN * EMBED_SIZE_L
        print(f'MHA : v.shape {v.shape} ')

        #break into n_heads
        q, k, v = [self.split_heads(t) for t in (q,k,v)]  # BS * SEQ_LEN * HEAD * EMBED_SIZE_P_HEAD
        print(f'MHA : After split_heads q.shape {q.shape} k.shape {k.shape} v.shape {v.shape}')
        q, k, v = [t.transpose(1,2) for t in (q,k,v)]  # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        print(f'MHA : After transpose(1,2) q.shape {q.shape} k.shape {k.shape} v.shape {v.shape}')

        #n_heads => attention => merge the heads => mix information
        scores = attention(q, k, v, mask, self.dropout) # BS * HEAD * SEQ_LEN * EMBED_SIZE_P_HEAD
        print(f'MHA : scores after attention {scores.shape}')
        scores = scores.transpose(1,2).contiguous().view(scores.shape[0], -1, self.out_dim) # BS * SEQ_LEN * EMBED_SIZE_L
        print(f'MHA : scores after view & transpose {scores.shape}')
        out = self.out(scores)  # BS * SEQ_LEN * EMBED_SIZE

        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, inp_dim, inner_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(inp_dim, inner_dim)
        self.linear2 = nn.Linear(inner_dim, inp_dim)
        self.dropout = nn.Dropout(dropout)
        print(f'FeedForward inp_dim inner_dim : {inp_dim} {inner_dim}')

    def forward(self, x):
        #inp => inner => relu => dropout => inner => inp
        print(f'FeedForward before changes x.shape {x.shape}')
        return self.linear2(self.dropout(F.relu(self.linear1(x))))

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, inner_transformer_size, inner_ff_size, dropout=0.1):
        super().__init__()
        self.mha = MultiHeadAttention(n_heads, inner_transformer_size, dropout)
        self.ff = FeedForward(inner_transformer_size, inner_ff_size, dropout)
        self.norm1 = nn.LayerNorm(inner_transformer_size)
        self.norm2 = nn.LayerNorm(inner_transformer_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x2 = self.norm1(x)
        print(f'Encoder Layer x2.shape after norm1 {x2.shape} x.shape {x.shape}')
        x = x + self.dropout1(self.mha(x2, mask=mask))
        print(f'Encoder Layer x.shape after mha {x.shape} mask {mask}')
        x2 = self.norm2(x)
        print(f'Encoder Layer x2.shape after norm2 {x2.shape} x.shape {x.shape}')
        x = x + self.dropout2(self.ff(x2))
        print(f'Encoder Layer x.shape after ff {x.shape} ')
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, n_code, n_heads, embed_size, inner_ff_size, n_embeddings, seq_len, dropout=.1):
        super().__init__()

        #model input
        self.embeddings = nn.Embedding(n_embeddings, embed_size)
        self.pe = PositionalEmbedding(embed_size, seq_len)

        #backbone
        encoders = []
        for i in range(n_code):
            encoders += [EncoderLayer(n_heads, embed_size, inner_ff_size, dropout)]
        self.encoders = nn.ModuleList(encoders)

        #language model
        self.norm = nn.LayerNorm(embed_size)
        self.linear = nn.Linear(embed_size, n_embeddings, bias=False)
        print(f'Transformer-init n_code {n_code} n_heads {n_heads} embed_size {embed_size} inner_ff_size {inner_ff_size} \
                n_embeddings {n_embeddings} seq_len {seq_len}')


    def forward(self, x):
        print(f'Transformer x.shape {x.shape}')
        x = self.embeddings(x)
        print(f'Transformer after self.embeddings x.shape {x.shape}')
        x = x + self.pe(x)
        print(f'Transformer after pe addition x.shape {x.shape} pe.shape {self.pe(x).shape}')
        for encoder in self.encoders:
            x = encoder(x)
        x = self.norm(x)
        x = self.linear(x)
        print(f'Transformer after self.linear x.shape {x.shape}')
        return x

In [ ]:
# Positional Embedding
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        pe.requires_grad = False
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        print(f'PositionalEmbedding self.pe.shape {self.pe.shape} x.shape {x.shape}')
        return self.pe[:,:x.size(1)] #x.size(1) = seq_len

In [ ]:
# =============================================================================
# Dataset
# =============================================================================
class SentencesDataset(Dataset):
    #Init dataset
    def __init__(self, sentences, vocab, seq_len):
        dataset = self

        dataset.sentences = sentences
        dataset.vocab = vocab + ['<ignore>', '<oov>', '<mask>']
        dataset.vocab = {e:i for i, e in enumerate(dataset.vocab)}
        dataset.rvocab = {v:k for k,v in dataset.vocab.items()}
        dataset.seq_len = seq_len

        #special tags
        dataset.IGNORE_IDX = dataset.vocab['<ignore>'] #replacement tag for tokens to ignore
        dataset.OUT_OF_VOCAB_IDX = dataset.vocab['<oov>'] #replacement tag for unknown words
        dataset.MASK_IDX = dataset.vocab['<mask>'] #replacement tag for the masked word prediction task
        print(f'len(sentences) len(vocab) seq_len : {len(sentences)} {len(vocab)} {seq_len}')
        print(f'dataset.IGNORE_IDX {dataset.IGNORE_IDX} dataset.OUT_OF_VOCAB_IDX {dataset.OUT_OF_VOCAB_IDX} ')
        print(f'dataset.MASK_IDX {dataset.MASK_IDX}')


    #fetch data
    def __getitem__(self, index, p_random_mask=0.15):
        print(f'index : {index}')
        dataset = self

        #while we don't have enough word to fill the sentence for a batch
        s = []
        while len(s) < dataset.seq_len:
            print(f' getitem len(s) < dataset.seq_len: {len(s)} dataset.seq_len {dataset.seq_len}')
            s.extend(dataset.get_sentence_idx(index % len(dataset)))
            print(f' getitem index % len(dataset) : {index % len(dataset)} - {len(dataset)}')
            print(f' getitem dataset.get_sentence_idx(index % len(dataset)) : {dataset.get_sentence_idx(index % len(dataset))}')
            print(f' getitem s value after extension : {s}')
            index += 1
        print(f' getitem after len(s) < dataset.seq_len : {len(s)} - {s}')

        #ensure that the sequence is of length seq_len
        s = s[:dataset.seq_len]
        print(f' getitem s after checking seq_len : {len(s)} - {s}')
        [s.append(dataset.IGNORE_IDX) for i in range(dataset.seq_len - len(s))] #PAD ok
        print(f' getitem s after IGNORE_IDX : {len(s)} - {s}')

        #apply random mask
        rand_rand = random.random()
        s = [(dataset.MASK_IDX, w) if rand_rand < p_random_mask else (w, dataset.IGNORE_IDX) for w in s]
        print(f' getitem rand_rand {rand_rand} p_random_mask {p_random_mask}')
        print(f' getitem s after apply random mask : {len(s)} - {s}')

        data_dict = {'input': torch.Tensor([w[0] for w in s]).long(),
                     'target': torch.Tensor([w[1] for w in s]).long()}

        print(f' getitem data_dict : {data_dict}')

        return {'input': torch.Tensor([w[0] for w in s]).long(),
                'target': torch.Tensor([w[1] for w in s]).long()}

    #return length
    def __len__(self):
        return len(self.sentences)

    #get words id
    def get_sentence_idx(self, index):
        dataset = self
        s = dataset.sentences[index]
        print(f'get_sentence_idx s gathering index - {index} : {len(s)} - {s}')
        s = [dataset.vocab[w] if w in dataset.vocab else dataset.OUT_OF_VOCAB_IDX for w in s]
        print(f'get_sentence_idx s after vocab lookup: {len(s)} - {s}')
        return s

In [ ]:
# =============================================================================
# Methods / Class
# =============================================================================
def get_batch(loader, loader_iter):
    try:
        batch = next(loader_iter)
    except StopIteration:
        loader_iter = iter(loader)
        batch = next(loader_iter)
    return batch, loader_iter

In [ ]:
# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 2
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

initializing..


In [ ]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
# pth = './BERT/training.txt'
pth = './BERT/small_training.txt'
sentences = open(pth).read().lower().split('\n')
print(f'sentences length - {len(sentences)}, {sentences[:2]}')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
print(f'sentences length - {len(sentences)}, {sentences[:2]}')
sentences = [[w for w in s if len(w)] for s in sentences]
print(f'sentences length - {len(sentences)}, {len(sentences[0])} {len(sentences[-1])} {sentences[:2]}')

loading text...
sentences length - 6, ['i would like to print my bert model summary (text classification). ', 'i know that for image classification we use summary.']
tokenizing sentences...
sentences length - 6, [['i', 'would', 'like', 'to', 'print', 'my', 'bert', 'model', 'summary', '', '(', 'text', 'classification', ')', '', '.', '', ''], ['i', 'know', 'that', 'for', 'image', 'classification', 'we', 'use', 'summary', '.', '']]
sentences length - 6, 14 0 [['i', 'would', 'like', 'to', 'print', 'my', 'bert', 'model', 'summary', '(', 'text', 'classification', ')', '.'], ['i', 'know', 'that', 'for', 'image', 'classification', 'we', 'use', 'summary', '.']]


In [ ]:
for s in sentences:
  print(len(s), s)

14 ['i', 'would', 'like', 'to', 'print', 'my', 'bert', 'model', 'summary', '(', 'text', 'classification', ')', '.']
10 ['i', 'know', 'that', 'for', 'image', 'classification', 'we', 'use', 'summary', '.']
9 ['what', 'dimensions', 'can', 'i', 'give', 'for', 'text', 'bert', '?']
31 ['in', 'this', 'python', 'tutorial', ',', 'we', 'will', 'learn', 'how', 'to', 'create', 'a', 'pytorch', 'model', 'summary', 'in', 'python', 'and', 'we', 'will', 'also', 'cover', 'different', 'examples', 'related', 'to', 'the', 'pytorch', 'model', 'summary', '.']
8 ['moreover', ',', 'we', 'will', 'cover', 'these', 'topics', '.']
0 []


In [ ]:
#3) create vocab if not already created
print('creating/loading vocab...')
pth = './BERT/vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')
print(f'vocab : {len(vocab)} {vocab[:5]}')

creating/loading vocab...
vocab : 23945 [',', '.', "'", 'the', 'and']


In [ ]:
#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)
print(f'kwargs : {kwargs}')

creating dataset...
len(sentences) len(vocab) seq_len : 6 23945 20
dataset.IGNORE_IDX 23945 dataset.OUT_OF_VOCAB_IDX 23946 
dataset.MASK_IDX 23947
kwargs : {'shuffle': True, 'drop_last': True, 'pin_memory': True, 'batch_size': 2}


In [ ]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

initializing model...
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
FeedForward inp_dim inner_dim : 128 512
Transformer-init n_code 8 n_heads 8 embed_size 128 inner_ff_size 512                 n_embeddings 23948 seq_len 20


In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device:", DEVICE)
model.to(DEVICE)

device: cuda


Transformer(
  (embeddings): Embedding(23948, 128)
  (pe): PositionalEmbedding()
  (encoders): ModuleList(
    (0): EncoderLayer(
      (mha): MultiHeadAttention(
        (linear): Linear(in_features=128, out_features=384, bias=True)
        (out): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): EncoderLayer(
      (mha): MultiHeadAttention(
        (linear): Linear(in_features=128, out_features=384, bias=True)
        (out): Linear(in_features=128, out_featu

In [ ]:
# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

initializing optimizer and loss...


In [ ]:
# =============================================================================
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 2
for it in range(n_iteration):
    print(f'iteration - it : {it}')
    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    print(f' masked_input : {len(masked_input)} - {masked_input}')
    print(f' masked_target : {len(masked_target)} - {masked_target}')

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)
    print(f' output : {output.shape}')
    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    print(f' output_v : {output_v.shape}')
    target_v = masked_target.view(-1,1).squeeze()
    print(f' target_v : {target_v.shape}')
    loss = loss_model(output_v, target_v)
    print(f' loss : {type(loss)} {loss.size()}')
    if it == 1:
      print(f'loss : {loss}')
    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_each == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()


training...
iteration - it : 0
index : 3
 getitem len(s) < dataset.seq_len: 0 dataset.seq_len 20
get_sentence_idx s gathering index - 3 : 31 - ['in', 'this', 'python', 'tutorial', ',', 'we', 'will', 'learn', 'how', 'to', 'create', 'a', 'pytorch', 'model', 'summary', 'in', 'python', 'and', 'we', 'will', 'also', 'cover', 'different', 'examples', 'related', 'to', 'the', 'pytorch', 'model', 'summary', '.']
get_sentence_idx s after vocab lookup: 31 - [13, 29, 23946, 23946, 0, 45, 36, 887, 75, 6, 3263, 9, 23946, 4731, 13220, 13, 23946, 4, 45, 36, 2061, 2034, 4783, 6696, 23946, 6, 3, 23946, 4731, 13220, 1]
 getitem index % len(dataset) : 3 - 6
get_sentence_idx s gathering index - 3 : 31 - ['in', 'this', 'python', 'tutorial', ',', 'we', 'will', 'learn', 'how', 'to', 'create', 'a', 'pytorch', 'model', 'summary', 'in', 'python', 'and', 'we', 'will', 'also', 'cover', 'different', 'examples', 'related', 'to', 'the', 'pytorch', 'model', 'summary', '.']
get_sentence_idx s after vocab lookup: 31 - [1

In [ ]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )


print('end')

saving embeddings...


NameError: ignored